In [ ]:
!pip install luna_quantum

# Luna Quantum SDK: Traveling Salesman Problem Demo

Welcome to the **Luna Quantum SDK** demonstration! This comprehensive notebook showcases how Luna Quantum makes quantum optimization accessible, powerful, and easy to use through a complete Traveling Salesman Problem (TSP) solution.

## 🚀 Why Luna Quantum?

Luna Quantum is a cutting-edge SDK that bridges the gap between classical and quantum computing, offering:

- **Unified Programming Model**: Write once, run on classical CPUs, quantum simulators, or real quantum hardware
- **Smart Abstractions**: Focus on problem modeling while Luna handles algorithm-specific transformations
- **Plug-and-Play Architecture**: Seamlessly switch between algorithms and backends without code changes
- **Production-Ready**: Enterprise-grade quantum cloud platform with secure token management

## 🎯 What this Notebook will Show

This demo highlights LunaSolves's key capabilities through a practical TSP implementation:

1. **Rapid Prototyping**: Use predefined optimization patterns for quick problem setup
2. **Advanced Modeling**: Build custom models with AqModels for complete control
3. **Hybrid Solving**: Compare classical and quantum algorithms on the same problem
4. **Seamless Deployment**: Run on classical machines, cloud simulators, or quantum hardware

## 📍 The Challenge

We'll solve the TSP for 4 German cities: **Berlin**, **Hamburg**, **Munich**, and **Bonn**. The goal is to find the shortest route that visits all cities exactly once and returns to the starting point—a perfect demonstration of Luna Quantum's optimization capabilities.

## 📋 Prerequisites

Before running this notebook, ensure you have:
- ✅ Luna Quantum SDK installed (`pip install luna-quantum`)
- ✅ D-Wave token set as environment variable: `DWAVE_TOKEN`
- ✅ All dependencies installed (see `pyproject.toml`)

**Need help with setup?** Check the project README for detailed instructions.

# LunaSolve Overview
![LunaSolve Overview](plots/LunaSolve.png)


# 🚀 The LunaSolve Workflow

LunaSolve follows a simple 6-step process:

![LunaSolve Flow](plots/LunaSolveFlow.png)

In [ ]:
# Imports for this notebook
from luna_quantum import Model,Variable, Vtype
from luna_quantum.translator import LpTranslator
import networkx as nx
import itertools
import os
from luna_quantum.solve.use_cases import TravellingSalesmanProblem
from luna_quantum import LunaSolve
from luna_quantum.solve import algorithms, backends
from luna_quantum.client.schemas import QpuToken
import getpass

# Use Case utils
from utils.plotting import plot_graph_on_map, plot_solution_tour
from utils.traveling_sales_man import calculate_distance_matrix, extract_tour_from_solution

# Log in to the Luna platform
if "LUNA_API_KEY" not in os.environ:
    # Prompt securely for the key if not already set
    os.environ["LUNA_API_KEY"] = getpass.getpass("Enter your Luna API key: ")

# 🚀 Quick Start Example

![LunaSolve Workflow](plots/FlowComplete.png)

Before diving into detailed modeling, let's see LunaSolve in action with a pre-built model:

In [ ]:
# 1. Load a pre-built model
m = LpTranslator.to_aq('tsp_model.lp') # Note: We'll build this model in the sections below

# 2. Select Algorithm
alg = algorithms.SimulatedAnnealing(num_reads=100)

# 3. / 4. / 5.  Compute optimization
job = alg.run(model=m)

In [ ]:
# 6. Analyze solution
solution = job.result()
solution.best()

In [ ]:
# Extract tour from solution
tour = extract_tour_from_solution(
    variable_names=solution.variable_names,
    sample_values=solution.best().sample,
    start_city='Berlin'
)

# Display the optimal tour
print(f"🎯 Optimal Tour:")
print(f"   → Route: {' → '.join(tour)}")

**⚠️ Note**:
The above is a preview of the complete workflow. We'll build the actual model step-by-step in the following sections.

# 🚀 LunaSolve in Action

## Step 1: Bringing a Use Case into the Platform

![LunaSolve Flow Model](plots/FlowModel.png)

### Method 1: Predefined Use Cases - Luna Quantum's Rapid Development Engine

Luna Quantum's predefined use cases allows a simplified optimization development for
common problem types.



#### 🔗 Graph Representation

We use NetworkX to create a mathematical representation of our TSP problem:

**Graph Structure:**
- **Nodes**: Cities with GPS coordinates as attributes
- **Edges**: All possible routes between cities with distance weights
- **Complete Graph**: Every city connects to every other city (K₄ graph)

In [ ]:
# First, we define the data for our TSP
cities = {
    "Berlin": (52.52, 13.405),
    "Hamburg": (53.5511, 9.9937),
    "Munich": (48.1351, 11.582),
    "Bonn": (50.7374, 7.0982),
}
distance_matrix = calculate_distance_matrix(cities)

# Create a graph
G = nx.Graph()

# Add nodes with their geographic coordinates
for city, (lat, lon) in cities.items():
    G.add_node(city, pos=(lon, lat))  # Note: NetworkX expects (x, y) = (lon, lat)

# Add edges
for city1, city2 in itertools.combinations(cities, 2):
    G.add_edge(city1, city2, weight=distance_matrix[city1][city2])

In [ ]:
map = plot_graph_on_map(G = G, cities = cities)
map

In [ ]:
# Instantiate Luna
ls = LunaSolve()
tsp_graph = nx.to_dict_of_dicts(G)

# Use predefined use case to define model
tsp = TravellingSalesmanProblem(graph=tsp_graph)
model = ls.model.create_from_use_case(name="predefined TSP", use_case=tsp)

print(ls.model.get_model(model.id))

### Method 2: AqModels - Luna's Advanced Custom Modeling Engine

**AqModels** represents Luna's core package for advanced optimization modeling.
This powerful framework combines the best practices of classical optimization with
cutting-edge quantum capabilities, providing flexibility and control.

#### 🌟 AqModels Features:
- **Single API** for both classical and quantum algorithm development
- **Automatic problem transformations** (LP → QUBO, QUBO → Ising, etc.)
- **Algorithm-agnostic modeling** - write once, solve anywhere

In [ ]:
from luna_quantum import quicksum
# Decide whether to build this model with or without constraints (QUBO)
constrained_model = False

# Traveling Salesman Problem (TSP) - Position-Based Formulation
# This implementation uses a position-based approach where each city is assigned
# to a specific position in the tour (except the starting city which is fixed)

# Initialize the model and fix starting city
travel_cities = list(cities.keys())
start_city = travel_cities.pop(0)
n_nodes = len(cities)

model = Model()
x = {}
with model.environment:
    # Decision Variables x[i, j] = 1 if city i is visited at position j in the tour, 0 otherwise
    for i in travel_cities:
        for j in range(1, n_nodes):
            x[i, j] = Variable(f"x_{i}_{j}", vtype=Vtype.Binary)

# Objective Function: Minimize total travel distance

# Distance from and to starting city
for i in travel_cities:
    model.objective += distance_matrix[start_city][i] * x[i, 1]
    model.objective += distance_matrix[i][start_city] * x[i, n_nodes-1]

# Distance between consecutive positions in the tour
for pos in range(1, n_nodes-1):
    for i, j in itertools.combinations(travel_cities, 2):
        # If city i is at position 'pos' AND city j is at position 'pos+1'
        # then add distance from i to j and its reverse
        model.objective += distance_matrix[i][j] * x[i, pos] * x[j, pos+1]
        model.objective += distance_matrix[j][i] * x[j, pos] * x[i, pos+1]

# Constraints as penalty terms in objective
penalty_term = float(distance_matrix.max().max()) *10

# Constraint 1: Each city (except the starting city) must be visited exactly once
for i in travel_cities:
    lhs = quicksum(x[i, j] for j in range(1, n_nodes))
    if constrained_model:
        model.add_constraint(lhs == 1, name=f"city_{i}_visited_once")
    else:
        model.objective += penalty_term * (lhs**2) + penalty_term - 2*lhs*penalty_term

# Constraint 2: # Each position (except position 0 which is fixed to start_city)
# must be occupied by exactly one city
for j in range(1, n_nodes):
    lhs = sum(x[i, j] for i in travel_cities)
    if constrained_model:
        model.add_constraint(lhs == 1, name=f"pos_{j}_occupied_once")
    else:
        model.objective += penalty_term * (lhs**2) + penalty_term - 2*lhs*penalty_term

# The final model
print(model)

# easily save model
LpTranslator.from_aq(model = model, filepath ='tsp_model.lp')

## Step 2: Seamless Multi-Algorithm Execution

![LunaSolve Flow Algorithm Selection](plots/FlowAlgorithm.png)

LunaSolves's execution model demonstrates the power of our plug-and-play architecture. Below, we solve the **exact same model** using two completely different approaches:

### 🔐 Authentication Setup

Before running optimization jobs, we need to set up authentication for quantum hardware access:

In [ ]:
# Log in to the Luna platform
if "DWAVE_TOKEN" not in os.environ:
    # Prompt securely for the key if not already set
    os.environ["DWAVE_TOKEN"] = getpass.getpass("Enter your DWAVE key: ")

In [ ]:
# For example, set up D-Wave quantum computing token
personal_qpu_token = ls.qpu_token.create(
    provider='dwave',
    name='my-dwave-token',
    token=os.environ.get("DWAVE_TOKEN"),  # Set this environment variable
    token_type="personal"
)

**⚠️ Important Setup Notes:**
- Set your D-Wave token: `export DWAVE_TOKEN="your_token_here"`
- Get tokens from [D-Wave Leap](https://cloud.dwavesys.com/leap/)
- Tokens are securely managed by Luna Quantum
- No credentials stored in code - uses environment variables

### 🔧 Backend and Algorithm Configuration

LunaSolve's flexible architecture allows you to choose different execution environments:

In [ ]:
from luna_quantum.client.schemas.qpu_token.qpu_token import PersonalQpuToken

# Classical simulation backend (no tokens required)
classic_backend = backends.DWave()

# Quantum hardware backend (requires D-Wave QPU token)
personal_qpu_token = PersonalQpuToken(name='my-dwave-token')
qpu_backend = backends.DWaveQpu(token=personal_qpu_token)

# Algorithm selection with different backends
tabu_search = algorithms.TabuSearch(num_reads=100, backend=classic_backend)
quantum_annealing = algorithms.QuantumAnnealing(num_reads=100, backend=qpu_backend)


**Algorithm Comparison:**

- **Tabu Search**: Classical heuristic, fast execution, good for development
- **Quantum Annealing**: Quantum algorithm, potential for better solutions, requires quantum hardware

## Step 3+4+5: Solving the Use Case

![LunaSolve Flow Compute](plots/FlowCompute.png)

### 🚀 Parallel Job Execution

LunaSolve supports running multiple optimization jobs concurrently:

In [ ]:
print("   → Classical Tabu Search")
tabu_job = tabu_search.run(model)

In [ ]:
print("   → Quantum Annealing (if token available)")
quantum_job = quantum_annealing.run(model)

In [ ]:
print("✅ Jobs submitted successfully!")
print("   → Job 1 ID:", tabu_job.id if hasattr(tabu_job, 'id') else 'Classical job')
print("   → Job 2 ID:", quantum_job.id if hasattr(quantum_job, 'id') else 'Quantum job')

### 📊 LunaSolve's Job Management



LunaSolve's result handling system provides sophisticated, async job management and
analytics capabilities that make working with optimization results intuitive and powerful.

⏳ Waiting for job completion


In [ ]:
tabu_solution = tabu_job.result()
print(f'\nTabu Search found solution with {tabu_solution.best().obj_value}')
print(tabu_solution)

In [ ]:
qa_solution = quantum_job.result()
print(f'\nQuantum annealing found solution with {qa_solution.best().obj_value}')
print(qa_solution)

## Step 6: Intelligent Solution Processing

![LunaSolve Flow Solution](plots/FlowSolution.png)
Luna Quantum transforms raw optimization results into actionable business insights through sophisticated post-processing capabilities.

### 🗺️ Tour Extraction and Visualization

Transform optimization results into a readable tour format:

In [ ]:
# we will continue using the tabu search solution
solution = tabu_solution
# solution = qa_solution

# Extract tour from solution
tour = extract_tour_from_solution(
    variable_names=solution.variable_names,
    sample_values=solution.best().sample,
    start_city=start_city
)

# Display the optimal tour
print(f"🎯 Optimal Tour:")
print(f"   → Route: {' → '.join(tour)}")

# Calculate total distance
total_distance = 0
for i in range(len(tour) - 1):
    total_distance += distance_matrix[tour[i]][tour[i + 1]]

print(f"   → Total Distance: {total_distance:.1f} km")

### 🗺️ Interactive Solution Visualization

Create an interactive map showing the optimal tour:


In [ ]:
# Generate interactive map with solution
solution_map = plot_solution_tour(
    G=G,
    tour=tour,
    cities=cities,
    distance_matrix=distance_matrix,
)

print("🗺️  Interactive solution map generated!")
print(f"   → Shows optimal tour: {' → '.join(tour[:-1])}")
print(f"   → Green line indicates optimal route")
print(f"   → Red markers show city locations")

# Display the map
solution_map

---

## 🌟 Luna Quantum SDK: Your Gateway to Quantum-Enhanced Optimization

### 🎯 What You've Experienced

This demo showcased Luna Quantum's complete optimization ecosystem through a practical TSP implementation:

✅ **Rapid Development** with predefined optimization patterns  
✅ **Advanced Modeling** using AqModels' flexible framework  
✅ **Multi-Algorithm Execution** comparing classical and quantum approaches  
✅ **Enterprise-Grade Infrastructure** with secure token management  
✅ **Intelligent Result Processing** from variables to business decisions  


### 🚀 Ready to Transform Your Optimization Challenges?

**Start your quantum optimization journey with Luna Quantum SDK today!**

---

*For more information, visit our documentation, explore additional examples, or contact our team for enterprise solutions.*